In [1]:
%cleanup -f

In [2]:
%%configure -f
{
    "numExecutors": 10,
    "driverMemory": "20g"
}

In [3]:
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.linalg import SparseVector, DenseVector, Vectors, VectorUDT
import numpy as np
from pyspark.sql.window import Window

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1673740422743_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1673740422743_0003,pyspark,idle,Link,Link,None,✔


In [5]:
@f.udf(returnType=VectorUDT())
def sparse_vector_sum(vectors):
    res = None
    for vec in vectors:
        if res is None:
            res = vec
        else:
            res = np.add(vec, res)
    return SparseVector(len(res), {k: v for k, v in enumerate(res) if v != 0})

@f.udf(returnType=ArrayType(FloatType()))
def to_dense(sparse_vector):
    v = DenseVector(sparse_vector)
    feature = [float(x) for x in v]
    feature.append(0.0)  # adding util placeholder
    return feature

@f.udf(returnType=ArrayType(FloatType()))
def to_dense_flat(sparse_vector):
    v = DenseVector(sparse_vector)
    feature = [float(x) for x in v]
    return feature

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Aggregate Cell and User Hourly

In [34]:
df = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/tokenised_ebmr_sample/dt=2023-01-10/')
df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------+------+--------------------+--------+--------------------+------+-------+
|    event_timestamp|event_type_cd|result|            loc_info|     TAC|             Subs_Id|prefix|matched|
+-------------------+-------------+------+--------------------+--------+--------------------+------+-------+
|2023-01-10 10:14:09|           15|     0|U_505_03_336__16283_|14869004|063d6b4f880e925a9...|   118|  false|
|2023-01-10 20:20:39|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a9...|   118|  false|
|2023-01-10 13:23:19|           15|     0|  U_505_03_336__840_|14869004|063d6b4f880e925a9...|   118|  false|
|2023-01-10 00:46:22|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a9...|   118|  false|
|2023-01-10 06:02:05|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a9...|   118|  false|
|2023-01-10 09:11:05|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a9...|   118|  false|
|2023-01-10 15:20:2

In [2]:
df.select('event_type_cd').dropDuplicates().show(50, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|event_type_cd|
+-------------+
|7            |
|26           |
|25           |
|9            |
|15           |
|23           |
|13           |
|14           |
|3            |
|30           |
|22           |
|0            |
|24           |
|16           |
|6            |
|1            |
|11           |
|8            |
|5            |
|10           |
|4            |
|12           |
|2            |
|28           |
+-------------+

In [35]:
df = df.withColumn(
    'hour', f.hour('event_timestamp')
).withColumn(
    'event_type_cd', f.col('event_type_cd').cast('int')
)
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------+------+--------------------+--------+--------------------+------+-------+----+
|    event_timestamp|event_type_cd|result|            loc_info|     TAC|             Subs_Id|prefix|matched|hour|
+-------------------+-------------+------+--------------------+--------+--------------------+------+-------+----+
|2023-01-10 10:14:09|           15|     0|U_505_03_336__16283_|14869004|063d6b4f880e925a9...|   118|  false|  10|
|2023-01-10 20:20:39|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a9...|   118|  false|  20|
|2023-01-10 13:23:19|           15|     0|  U_505_03_336__840_|14869004|063d6b4f880e925a9...|   118|  false|  13|
|2023-01-10 00:46:22|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a9...|   118|  false|   0|
|2023-01-10 06:02:05|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a9...|   118|  false|   6|
|2023-01-10 09:11:05|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a

In [6]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3198939406

In [36]:
encoder = OneHotEncoder(inputCol='event_type_cd', outputCol='event_type_encode')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
encoder = encoder.fit(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df = encoder.transform(df)
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------+------+--------------------+--------+--------------------+------+-------+----+-----------------+
|    event_timestamp|event_type_cd|result|            loc_info|     TAC|             Subs_Id|prefix|matched|hour|event_type_encode|
+-------------------+-------------+------+--------------------+--------+--------------------+------+-------+----+-----------------+
|2023-01-10 10:14:09|           15|     0|U_505_03_336__16283_|14869004|063d6b4f880e925a9...|   118|  false|  10|  (30,[15],[1.0])|
|2023-01-10 20:20:39|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a9...|   118|  false|  20|  (30,[15],[1.0])|
|2023-01-10 13:23:19|           15|     0|  U_505_03_336__840_|14869004|063d6b4f880e925a9...|   118|  false|  13|  (30,[15],[1.0])|
|2023-01-10 00:46:22|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a9...|   118|  false|   0|  (30,[15],[1.0])|
|2023-01-10 06:02:05|           15|     0|  U_505_03_336__640_|14869004|063d

In [40]:
event_feature_data = df.filter(f.col('Subs_Id').isNotNull()).groupBy('Subs_Id', 'loc_info', 'hour').agg(
    sparse_vector_sum(f.collect_list('event_type_encode')).alias('event_code_feature')
)
# event_feature_data.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [173]:
# event_feature_data.write.mode('overwrite').parquet(
#     's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/ebmr_event_feature/dt=2023-01-10/')
event_feature_data = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/ebmr_event_feature/dt=2023-01-10/')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [174]:
event_feature_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Subs_Id: string (nullable = true)
 |-- loc_info: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- event_code_feature: vector (nullable = true)

In [175]:
event_feature_data.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

584027513

In [44]:
%cleanup -f

## Further Graph Formation

In [176]:
event_feature_data = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/ebmr_event_feature/dt=2023-01-10/')
event_feature_data.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------------------+-------------------------+----+-----------------------------------------+
|Subs_Id                                                         |loc_info                 |hour|event_code_feature                       |
+----------------------------------------------------------------+-------------------------+----+-----------------------------------------+
|0000747716c301fc203c8afa7252ce3526c9f72b41c063e58b845e5218fa9f0f|L_505_03__50533__13057316|20  |(30,[13],[3.0])                          |
|00011c44a7928fdfd3e8f2130815b53e96c9093cd1d0e7d07796023479f8c812|U_505_03_294__59501_     |4   |(30,[2],[1.0])                           |
|0001a038d111e7553e79893ee1231768b7b4e2a5d8f534f3fb1a6dd584bafd92|U_505_03_345__35178_     |5   |(30,[2],[1.0])                           |
|0002610ff58f0c3afd1e873d3dbb8bdcce55bad2461e1cf47ea526004d45b106|L_505_03__20234__5805089 |16  |(30,[7,8,12,13,16],[1.0,1.0,1.0,1.0,2.0])|
|0002949939f1a71fb62

In [49]:
event_feature_data.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

584027513

In [95]:
# cell info
@f.udf(returnType=StringType())
def resolve_cell_name(cell_name):
    """
    resolve cell name to match with EDS event data
    """
    try:
        split_parts = cell_name.split("_")
        m2000id = split_parts[0]
        site_name = split_parts[1]
        technology = split_parts[3]
        cell_number = split_parts[4]
        return f"{m2000id}_{site_name}_{technology}_{cell_number}"
    except Exception as _:
        return cell_name

locglbl = spark.read.parquet(
    's3://kl-prod-bi-analy-data-extract/features/kl-prod/ICE/EDS_locglbl_v3_extract/part_dt=2023-01-11/'
)
cell_lookup = spark.read.parquet(
    's3://kl-prod-bi-analy-data-extract/features/kl-prod/ICE/TD_cell_look_up/part_dt=2023-01-11/'
)
site_df = spark.read.parquet(
    's3://kl-prod-bi-analy-data-extract/features/kl-prod/ICE/TD_site_table/part_dt=2023-01-11/'
)

cell_info = (
    locglbl.join(
        cell_lookup.filter(
            f.col("Radio_Type_Cd").isin(["L", "U", "N"])
        ),  # ignore 3G cells. ie. 'U'. Ignores 5G for test. ie 'N'
        on=locglbl.glbl_loc_id == cell_lookup.Glbl_loc_Id,
        how="inner",
    )
    .withColumn("m2000_id", f.split(f.col("Cell_Name"), "_").getItem(0))
    .join(
        site_df,  # .filter(f.col("sa6_state_nm") == sa6_nm),
        on="m2000_id",
        how="inner",
    )
    .withColumn("cell_name_adjusted", resolve_cell_name(f.col("Cell_Name")))
    .select(
        locglbl.glbl_loc,
        locglbl.glbl_loc_id,
        f.col("cell_name_adjusted").alias("cell_name"),
        locglbl.mcc,
        locglbl.mnc,
        site_df.state
    )
)
cell_info.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------+----------------------------+-----------------------------+---+---+---------------+
|glbl_loc                    |glbl_loc_id                 |cell_name                    |mcc|mnc|state          |
+----------------------------+----------------------------+-----------------------------+---+---+---------------+
|U_505_03_441__48182_        |U_505_03_00441_48182_48182  |477366_Woolloongabba_U09A_3  |505|03 |Queensland     |
|L_505_03__20248__005136919  |L_505_03_20248_|_005136919  |202172_Ingleburn_L18B_3      |505|03 |New South Wales|
|N_505_03__40442__00748322878|N_505_03_40442_|_00748322878|477064_Chermside_N07C_2      |505|03 |Queensland     |
|U_505_03_238__60696_        |U_505_03_00238_60696_60696  |201537_AnnaBay_U21A_3        |505|03 |New South Wales|
|L_505_03__30341__8508723    |L_505_03_30341_|_008508723  |303113_Anglesea_L08B_1       |505|03 |Victoria       |
|U_505_03_538__40768_        |U_505_03_00538_40768_40768  |505215_Echunga_U09A_2        

In [91]:
# cei df
cei_df = spark.read.parquet('s3://kl-prod-bi-analy-data-extract/features/kl-prod/Mobility/customer_exp_index_v3/part_dt=2023-01-11/')
cei_df =cei_df.withColumnRenamed('subs_id', 'Subs_Id')
cei_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+--------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+--------------+----------------+------------------+-----------------+------------+------------------------+------------------------+------------------------+----------------

In [113]:
# get 1 million customer graphs
valid_subs = cei_df.select('Subs_Id').dropDuplicates()
valid_subs.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4530307

In [114]:
nsw_event_df = event_feature_data.join(
    cell_info.filter(f.col('state') == 'New South Wales'),
    on=event_feature_data.loc_info==cell_info.glbl_loc,
    how='inner'
).drop('glbl_loc', 'glbl_loc_id', 'loc_info', 'mcc', 'mnc').join(
    valid_subs,
    on='Subs_Id',
    how='inner'
)
nsw_event_df.show()
nsw_event_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+--------------------+--------------------+---------------+
|  Subs_Id|hour|  event_code_feature|           cell_name|          state|
+---------+----+--------------------+--------------------+---------------+
|282239187|   6|(30,[9,12,13,16],...|202154_NorthBondi...|New South Wales|
|317968313|  17|     (30,[13],[7.0])|201530_DeanPark_L...|New South Wales|
|318190067|   0|     (30,[13],[1.0])|201530_DeanPark_L...|New South Wales|
|318237737|  13|     (30,[13],[1.0])|201530_DeanPark_L...|New South Wales|
|318242619|   7|      (30,[7],[2.0])|201530_DeanPark_L...|New South Wales|
|318245227|   7|     (30,[13],[2.0])|202502_Greenacre2...|New South Wales|
|336293551|  23|     (30,[13],[1.0])|201658_BankstownC...|New South Wales|
|340160040|  17|(30,[7,8],[1.0,1.0])|202154_NorthBondi...|New South Wales|
|344685444|   3|     (30,[13],[2.0])|201658_BankstownC...|New South Wales|
|345846207|   8|(30,[7,13],[2.0,1...|202154_NorthBondi...|New South Wales|
|346898345|   6|      (30

In [154]:
# read the nsw data only
# nsw_event_df.write.mode('overwrite').parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/nsw_event_sample/dt=2023-01-10/')
nsw_event_df = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/nsw_event_sample/dt=2023-01-10/')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
@f.udf(returnType=StringType())
def resolve_sector_name(cell_name):
    """
    resolve cell name to match with EDS event data
    """
    try:
        split_parts = cell_name.split("_")
        m2000id = split_parts[0]
        site_name = split_parts[1]
        cell_number = split_parts[3]
        return f"{m2000id}_{site_name}_{cell_number}"
    except Exception as _:
        return cell_name
nsw_event_df = nsw_event_df.withColumn('sector_name', resolve_sector_name(f.col('cell_name')))
nsw_event_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+--------------------+--------------------+---------------+--------------------+
|  Subs_Id|hour|  event_code_feature|           cell_name|          state|         sector_name|
+---------+----+--------------------+--------------------+---------------+--------------------+
|282239187|  10|      (30,[7],[1.0])|202154_NorthBondi...|New South Wales|202154_NorthBondi...|
|317969389|   5|(30,[8,13],[1.0,4...|201530_DeanPark_L...|New South Wales|   201530_DeanPark_2|
|318450485|   2|     (30,[13],[3.0])|201658_BankstownC...|New South Wales|201658_BankstownC...|
|318484546|  22|     (30,[13],[1.0])|202154_NorthBondi...|New South Wales|202154_NorthBondi...|
|318525279|  18|     (30,[13],[1.0])|228310_NeutralBay...|New South Wales|228310_NeutralBay...|
|318562842|  15|     (30,[13],[1.0])|202154_NorthBondi...|New South Wales|202154_NorthBondi...|
|347769196|   0|     (30,[13],[3.0])|202154_NorthBondi...|New South Wales|202154_NorthBondi...|
|352584715|  16|      (30,[7],[2.0])|202

In [156]:
nsw_event_df.filter(
    f.col('sector_name') == '227403_Rockdale_2'
).select('cell_name', 'sector_name').dropDuplicates().show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+-----------------+
|cell_name             |sector_name      |
+----------------------+-----------------+
|227403_Rockdale_L21H_2|227403_Rockdale_2|
|227403_Rockdale_L18A_2|227403_Rockdale_2|
|227403_Rockdale_L18B_2|227403_Rockdale_2|
|227403_Rockdale_L08A_2|227403_Rockdale_2|
+----------------------+-----------------+

In [157]:
# utilisation df
utilisation = spark.read.parquet('s3://kl-prod-bi-analy-data-extract/features/kl-prod/ICE/RAN_util_hr_multibeam/part_dt=2023-01-11/')
utilisation = utilisation.withColumn('hour', f.hour('starttime'))
utilisation.printSchema()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- starttime: timestamp (nullable = true)
 |-- sector_name: string (nullable = true)
 |-- multibeam: long (nullable = true)
 |-- m2000_id: string (nullable = true)
 |-- prb_dl_util_pct: double (nullable = true)
 |-- weighted_prb_dl_util_pct: double (nullable = true)
 |-- prb_dl_util_pct_multibeam_weighted: double (nullable = true)
 |-- utilisation: double (nullable = true)
 |-- chmeas_prb_dl_used_avg: double (nullable = true)
 |-- mean_prb_avail_pdsch: decimal(38,0) (nullable = true)
 |-- traf_activeuser_dl_avg: double (nullable = true)
 |-- thrp_dl_qci_8: decimal(38,4) (nullable = true)
 |-- thrp_ul_qci_8: decimal(38,4) (nullable = true)
 |-- dlpayload: decimal(38,0) (nullable = true)
 |-- ulpayload: decimal(38,0) (nullable = true)
 |-- hour: integer (nullable = true)

In [158]:
# utilisation_nokia = spark.read.parquet('s3://kl-prod-bi-analy-data-extract/features/kl-prod/ICE/RAN_util_hr_multibeam_nokia/part_dt=2023-01-11/')
# utilisation_nokia.printSchema()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [159]:
utilisation.select(f.max('starttime'), f.min('starttime')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+
|     max(starttime)|     min(starttime)|
+-------------------+-------------------+
|2023-01-10 23:00:00|2023-01-10 00:00:00|
+-------------------+-------------------+

In [160]:
nsw_event_df = nsw_event_df.join(
    utilisation.select('hour', 'sector_name', 'utilisation'),
    on=['hour', 'sector_name'],
    how='inner'
)

nsw_event_df.show()
nsw_event_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+---------+--------------------+--------------------+---------------+------------------+
|hour|         sector_name|  Subs_Id|  event_code_feature|           cell_name|          state|       utilisation|
+----+--------------------+---------+--------------------+--------------------+---------------+------------------+
|   5|   201530_DeanPark_2|317969389|(30,[8,13],[1.0,4...|201530_DeanPark_L...|New South Wales|             25.32|
|  18|228310_NeutralBay...|318525279|     (30,[13],[1.0])|228310_NeutralBay...|New South Wales|             31.33|
|  18|201989_Shellharbo...|355583372|    (30,[13],[21.0])|201989_Shellharbo...|New South Wales|19.498461538461537|
|  14|   227970_Terrigal_2|357494257|(30,[8,13],[1.0,3...|227970_Terrigal_L...|New South Wales| 15.76923076923077|
|  12|   201530_DeanPark_2|358858769|     (30,[13],[5.0])|201530_DeanPark_L...|New South Wales| 72.79384615384615|
|  14|   227403_Rockdale_2|360369947|     (30,[13],[1.0])|227403_Rockdale_L...|N

In [161]:
nsw_event_df = nsw_event_df.withColumn('hour', f.lpad('hour', 2, '0'))
nsw_event_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+---------+--------------------+--------------------+---------------+------------------+
|hour|         sector_name|  Subs_Id|  event_code_feature|           cell_name|          state|       utilisation|
+----+--------------------+---------+--------------------+--------------------+---------------+------------------+
|  05|   201530_DeanPark_2|317969389|(30,[8,13],[1.0,4...|201530_DeanPark_L...|New South Wales|             25.32|
|  18|228310_NeutralBay...|318525279|     (30,[13],[1.0])|228310_NeutralBay...|New South Wales|             31.33|
|  18|201989_Shellharbo...|355583372|    (30,[13],[21.0])|201989_Shellharbo...|New South Wales|19.498461538461537|
|  14|   227970_Terrigal_2|357494257|(30,[8,13],[1.0,3...|227970_Terrigal_L...|New South Wales| 15.76923076923077|
|  12|   201530_DeanPark_2|358858769|     (30,[13],[5.0])|201530_DeanPark_L...|New South Wales| 72.79384615384615|
|  14|   227403_Rockdale_2|360369947|     (30,[13],[1.0])|227403_Rockdale_L...|N

In [162]:
nsw_event_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- hour: string (nullable = true)
 |-- sector_name: string (nullable = true)
 |-- Subs_Id: string (nullable = true)
 |-- event_code_feature: vector (nullable = true)
 |-- cell_name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- utilisation: double (nullable = true)

In [180]:
# # trace_id
# trace_id = nsw_event_df.select('Subs_Id', 'hour').withColumn(
#     'trace_id', 
#     f.row_number().over(Window().orderBy('Subs_Id', 'hour'))
# )
# trace_id.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [168]:
flat_nsw = nsw_event_df.withColumn(
    'trace_id', 
    f.rank().over(Window().orderBy('Subs_Id', 'hour'))
)
flat_nsw.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+----------+--------------------+--------------------+---------------+------------------+--------+
|hour|         sector_name|   Subs_Id|  event_code_feature|           cell_name|          state|       utilisation|trace_id|
+----+--------------------+----------+--------------------+--------------------+---------------+------------------+--------+
|  01|202583_RandwickCe...|2100006261|     (30,[13],[1.0])|202583_RandwickCe...|New South Wales|10.772307692307692|       1|
|  04|202583_RandwickCe...|2100006261|      (30,[8],[1.0])|202583_RandwickCe...|New South Wales|12.544615384615383|       2|
|  07|202583_RandwickCe...|2100006261|     (30,[13],[1.0])|202583_RandwickCe...|New South Wales|24.883076923076924|       3|
|  10|202583_RandwickCe...|2100006261|      (30,[8],[1.0])|202583_RandwickCe...|New South Wales|25.113846153846154|       4|
|  12|202583_RandwickCe...|2100006261|     (30,[13],[1.0])|202583_RandwickCe...|New South Wales|43.815384615384616|       5|


In [177]:
flat_nsw.groupBy('Subs_Id', 'hour').count().orderBy(f.col('count').desc()).limit(10).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+-----+
|  Subs_Id|hour|count|
+---------+----+-----+
|390914401|  10|   89|
|388852765|  11|   86|
|372354612|  11|   86|
|383291285|  11|   85|
|391419389|  10|   79|
|362142029|  09|   78|
|391232921|  13|   75|
|375549430|  10|   74|
|390333449|  09|   73|
|377318354|  16|   71|
+---------+----+-----+

In [178]:
flat_nsw.filter((f.col('Subs_Id') == '390914401')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+---------+--------------------+--------------------+---------------+------------------+--------+
|hour|         sector_name|  Subs_Id|  event_code_feature|           cell_name|          state|       utilisation|trace_id|
+----+--------------------+---------+--------------------+--------------------+---------------+------------------+--------+
|  03|201647_BondiJunct...|390914401|      (30,[8],[1.0])|201647_BondiJunct...|New South Wales| 9.298461538461538|48387731|
|  03|201647_BondiJunct...|390914401|      (30,[8],[2.0])|201647_BondiJunct...|New South Wales| 9.298461538461538|48387731|
|  05| 240178_WynyardRep_3|390914401|(30,[7,8],[1.0,1.0])|240178_WynyardRep...|New South Wales| 18.70153846153846|48387733|
|  05|202107_NthSydMcLa...|390914401|(30,[7,8],[2.0,1.0])|202107_NthSydMcLa...|New South Wales|20.273846153846154|48387733|
|  05|202558_HarbourBri...|390914401|      (30,[7],[1.0])|202558_HarbourBri...|New South Wales|10.732307692307694|48387733|
|  05|22

In [179]:
flat_nsw.write.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/nsw_event_trace/dt=2023-01-10/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [181]:
%cleanup -f

## Features

In [148]:
flat_nsw = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/nsw_event_trace/dt=2023-01-10/')
flat_nsw.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+----------+--------------------+--------------------+---------------+------------------+--------+
|hour|         sector_name|   Subs_Id|  event_code_feature|           cell_name|          state|       utilisation|trace_id|
+----+--------------------+----------+--------------------+--------------------+---------------+------------------+--------+
|  01|202583_RandwickCe...|2100006261|     (30,[13],[1.0])|202583_RandwickCe...|New South Wales|10.772307692307692|       1|
|  04|202583_RandwickCe...|2100006261|      (30,[8],[1.0])|202583_RandwickCe...|New South Wales|12.544615384615383|       2|
|  07|202583_RandwickCe...|2100006261|     (30,[13],[1.0])|202583_RandwickCe...|New South Wales|24.883076923076924|       3|
|  10|202583_RandwickCe...|2100006261|      (30,[8],[1.0])|202583_RandwickCe...|New South Wales|25.113846153846154|       4|
|  12|202583_RandwickCe...|2100006261|     (30,[13],[1.0])|202583_RandwickCe...|New South Wales|43.815384615384616|       5|


In [6]:
subs_features = flat_nsw.groupBy('Subs_Id', 'hour').agg(
    sparse_vector_sum(f.collect_list('event_code_feature')).alias('event_code_feature')
)

subs_features.show()
subs_features.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+--------------------+
|   Subs_Id|hour|  event_code_feature|
+----------+----+--------------------+
|2100006360|  23|(30,[7,13],[1.0,3...|
|2100006525|  04|    (30,[13],[20.0])|
|2100006553|  22|     (30,[13],[2.0])|
|2100006566|  01|(30,[12,13],[1.0,...|
|2100006758|  14|(30,[7,13],[2.0,1...|
|2100006787|  09|(30,[7,13],[1.0,1...|
|2100006787|  10|(30,[7,13],[2.0,8...|
|2100006952|  02|     (30,[13],[4.0])|
|2100007034|  07|(30,[7,12,13],[1....|
|2100007302|  08|      (30,[8],[1.0])|
|2100007307|  06|     (30,[13],[3.0])|
|2100007369|  07|(30,[7,8,13],[1.0...|
|2100007428|  17|     (30,[13],[1.0])|
|2100007534|  16|      (30,[7],[2.0])|
|2100007889|  17|(30,[7,8,9,10,13,...|
|2100008095|  16|(30,[12,13],[1.0,...|
|2100008116|  08|(30,[7,8,11,12,13...|
|2100008116|  18|(30,[7,8,11,12,13...|
|2100008209|  15|(30,[7,13],[6.0,4...|
|2100008421|  00|     (30,[13],[1.0])|
+----------+----+--------------------+
only showing top 20 rows

15006085

In [7]:
cell_features = flat_nsw.groupBy('cell_name', 'hour').agg(
    sparse_vector_sum(f.collect_list('event_code_feature')).alias('event_code_feature')
)

cell_features.show()
cell_features.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+--------------------+
|           cell_name|hour|  event_code_feature|
+--------------------+----+--------------------+
|200027_MLCCentre_...|  07|(30,[5,6,7,8,9,10...|
|200031_ChifleyTwr...|  14|(30,[5,6,7,8,9,10...|
|200031_ChifleyTwr...|  07|(30,[0,1,2,4,15],...|
|200032_GrosvenorP...|  15|(30,[5,6,7,8,9,10...|
|200032_GrosvenorP...|  12|(30,[5,7,8,9,10,1...|
|200032_GrosvenorP...|  07|(30,[6,7,8,9,12,1...|
|200033_Gateway_U2...|  10|(30,[4,15],[1.0,1...|
|200036_WinterGdnP...|  22|(30,[2,4,15],[1.0...|
|200038_MaritimeCe...|  07|(30,[7,8,11,13,16...|
|200040_MacquarieS...|  22|(30,[6,7,8,9,10,1...|
|200040_MacquarieS...|  17|(30,[5,6,7,8,9,10...|
|200040_MacquarieS...|  05|(30,[7,8,10,13],[...|
|200040_MacquarieS...|  05|(30,[6,7,8,11,12,...|
|200040_MacquarieS...|  11|(30,[6,7,8,9,10,1...|
|200046_StarCityHo...|  14|(30,[7,8,10,13,16...|
|200046_StarCityHo...|  10|(30,[7,8,9,10,13,...|
|200046_StarCityHo...|  18|(30,[4,15],[1.0,1...|
|200048_AAPTWorldS..

In [8]:
sector_features = flat_nsw.groupBy('sector_name', 'hour').agg(
    sparse_vector_sum(f.collect_list('event_code_feature')).alias('event_code_feature')
)

sector_features.show()
sector_features.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+--------------------+
|         sector_name|hour|  event_code_feature|
+--------------------+----+--------------------+
|200028_AustraliaS...|  17|(30,[0,1,2,4,5,6,...|
|  200034_Metcentre_3|  10|(30,[0,1,2,4,5,6,...|
|200036_WinterGdnP...|  12|(30,[5,6,7,8,9,10...|
|200046_StarCityHo...|  10|(30,[4,5,6,7,8,9,...|
|200130_MGLBuilding_1|  23|(30,[7,8],[2.0,1.0])|
|200130_MGLBuilding_3|  00|(30,[7,8,9,10,11,...|
| 200132_WavertonRS_2|  19|(30,[5,6,7,8,9,10...|
|200140_DarlingPar...|  02|(30,[7,13],[4.0,1...|
|200216_CastlerghK...|  14|(30,[5,6,7,8,9,10...|
|200219_AuroraPlace_4|  09|(30,[7,8,9,10,11,...|
|200226_FairfaxMed...|  08|(30,[4,7,8,9,10,1...|
|200229_NthSydneyR...|  08|(30,[2,5,6,7,8,9,...|
|200238_SheratonOn...|  16|(30,[2,4,5,6,7,8,...|
|200240_Darlingprk...|  05|(30,[6,7,8,9,10,1...|
|200243_126Phillip...|  06|(30,[5,7,8,13],[2...|
|200243_126Phillip...|  19|(30,[5,7,8,9,10,1...|
|200246_TownHallWy...|  12|(30,[2,5,6,7,8,9,...|
|200246_TownHallWy..

In [9]:
flat_nsw.groupBy('sector_name', 'hour').max('utilisation').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+------------------+
|         sector_name|hour|  max(utilisation)|
+--------------------+----+------------------+
|227899_IngleburnW...|  01|16.572307692307692|
|    202525_Leumeah_2|  11|37.723076923076924|
|   202807_Woolwich_2|  16| 49.12615384615385|
|201799_BadgeryCre...|  19| 19.27076923076923|
|    227623_Lugarno_3|  18|              9.89|
|     202975_Rhodes_3|  14| 66.24615384615385|
|201872_Charlestown_2|  09| 19.92923076923077|
|201901_CulburraBe...|  15|             31.48|
|    240136_Gorokan_3|  12| 81.07384615384615|
|    202533_JLTower_2|  04| 6.156923076923078|
|240086_M7QuakersH...|  08|            41.505|
|201998_ClaymoreCe...|  16|            40.148|
|202347_NewCastleE...|  06| 23.53538461538461|
|    202512_Revesby_3|  15| 6.772307692307693|
|228040_RootyHillM7_3|  14| 32.18153846153846|
|201690_GlenbrookW...|  23| 4.406153846153846|
| 227994_PenrithCBD_3|  17| 61.07076923076923|
|      202207_Umina_1|  05|10.363636363636363|
|227476_Marou

In [150]:
flat_nsw.select('trace_id').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|trace_id|
+--------+
|       1|
|       2|
|       3|
|       4|
|       5|
|       6|
|       7|
|       8|
|       9|
|      10|
|      11|
|      12|
|      13|
|      14|
|      15|
|      16|
|      17|
|      18|
|      19|
|      20|
+--------+
only showing top 20 rows

In [153]:
flat_nsw.select('trace_id').orderBy(f.col('trace_id').desc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|trace_id|
+--------+
|54855701|
|54855700|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
|54855678|
+--------+
only showing top 20 rows

In [152]:
flat_nsw.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

54855701

In [158]:
# trace_id_map = flat_nsw.select('Subs_Id', 'hour').withColumn(
#     'trace_id', f.row_number().over(Window.orderBy('Subs_Id', 'hour'))
# ).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Trace Info

In [159]:
# cei df
cei_df = spark.read.parquet('s3://kl-prod-bi-analy-data-extract/features/kl-prod/Mobility/customer_exp_index_v3/part_dt=2023-01-11/')
cei_df =cei_df.withColumnRenamed('subs_id', 'Subs_Id').withColumnRenamed('local_hr', 'hour')
cei_df = cei_df.withColumn('hour', f.lpad('hour', 2, '0'))
cei_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+-------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+--------------+----------------+------------------+-----------------+------------+------------------------+------------------------+------------------------+-----------------------

In [12]:
cei_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Subs_Id: string (nullable = true)
 |-- local_date: date (nullable = true)
 |-- hour: string (nullable = true)
 |-- sector_name_1: string (nullable = true)
 |-- sn_1_numevent: long (nullable = true)
 |-- sn_1_numLTEevnt: long (nullable = true)
 |-- sn_1_numIRRegevnt: long (nullable = true)
 |-- sn_1_utilisation: float (nullable = true)
 |-- sector_name_2: string (nullable = true)
 |-- sn_2_numevent: long (nullable = true)
 |-- sn_2_numLTEevnt: long (nullable = true)
 |-- sn_2_numIRRegevnt: long (nullable = true)
 |-- sn_2_utilisation: float (nullable = true)
 |-- sector_name_3: string (nullable = true)
 |-- sn_3_numevent: long (nullable = true)
 |-- sn_3_numLTEevnt: long (nullable = true)
 |-- sn_3_numIRRegevnt: long (nullable = true)
 |-- sn_3_utilisation: float (nullable = true)
 |-- sector_name_4: string (nullable = true)
 |-- sn_4_numevent: long (nullable = true)
 |-- sn_4_numLTEevnt: long (nullable = true)
 |-- sn_4_numIRRegevnt: long (nullable = true)
 |-- sn_4_utilisati

In [60]:
cei_df.filter(f.col('local_date') == '2023-01-09').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+--------------------+-------------+---------------+-----------------+----------------+-------------+-------------+---------------+-----------------+----------------+-------------+-------------+---------------+-----------------+----------------+-------------+-------------+---------------+-----------------+----------------+-------------+-------------+---------------+-----------------+----------------+-------------+-------------+---------------+-----------------+----------------+--------------+--------------+----------------+------------------+-----------------+------------+------------------------+------------------------+------------------------+------------------------+------------------------+-----------

In [13]:
cei_df.select('local_date', 'hour').dropDuplicates().orderBy('hour').show(50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+
|local_date|hour|
+----------+----+
|2023-01-10|  00|
|2023-01-10|  01|
|2023-01-10|  02|
|2023-01-10|  03|
|2023-01-10|  04|
|2023-01-10|  05|
|2023-01-10|  06|
|2023-01-10|  07|
|2023-01-10|  08|
|2023-01-10|  09|
|2023-01-10|  10|
|2023-01-10|  11|
|2023-01-10|  12|
|2023-01-10|  13|
|2023-01-10|  14|
|2023-01-10|  15|
|2023-01-10|  16|
|2023-01-10|  17|
|2023-01-10|  18|
|2023-01-10|  19|
|2023-01-10|  20|
|2023-01-10|  21|
|2023-01-10|  22|
|2023-01-09|  23|
+----------+----+

In [160]:
valid_subs_event = flat_nsw.select('Subs_Id', 'hour').dropDuplicates()
valid_subs_cei = cei_df.select('Subs_Id', 'hour').dropDuplicates()

valid_subs = valid_subs_event.join(valid_subs_cei, on=['Subs_Id', 'hour'], how='inner')
valid_subs.persist()
valid_subs.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14965018

In [161]:
valid_subs.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+
|  Subs_Id|hour|
+---------+----+
|279225784|  22|
|281042075|  07|
|283432783|  18|
|283475317|  19|
|283511381|  10|
|283666116|  07|
|317993537|  13|
|318014302|  00|
|318028999|  12|
|318091752|  03|
|318092721|  14|
|318271986|  17|
|318348865|  15|
|318390814|  20|
|318451913|  03|
|318485315|  19|
|318510507|  13|
|318596774|  18|
|318605389|  10|
|318610542|  15|
+---------+----+
only showing top 20 rows

In [162]:
valid_cei_df = cei_df.join(valid_subs, on=['Subs_Id', 'hour'], how='inner').select(
    'Subs_Id', 'hour', 'customer_experience_index'
).filter(f.col('Subs_Id').isNotNull())
valid_cei_df.show()
valid_cei_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+-------------------------+
|   Subs_Id|hour|customer_experience_index|
+----------+----+-------------------------+
|2100006261|  17|                     86.0|
|2100006328|  10|                     27.0|
|2100006348|  14|                     22.0|
|2100006392|  15|                     12.0|
|2100006510|  03|                     91.0|
|2100006646|  01|                     98.0|
|2100006671|  16|                     92.0|
|2100006682|  03|                     29.0|
|2100006682|  08|                     42.0|
|2100006733|  06|                     79.0|
|2100006777|  08|                     84.0|
|2100006820|  01|                     89.0|
|2100006880|  07|                     81.0|
|2100006894|  09|                     82.0|
|2100006923|  19|                     85.0|
|2100006934|  10|                     86.0|
|2100006952|  23|                    100.0|
|2100006978|  20|                     91.0|
|2100006982|  01|                     99.0|
|2100007012|  15|               

In [174]:
valid_flat_nsw = flat_nsw.join(
    valid_subs, on=['Subs_Id', 'hour'], how='inner'
)
valid_flat_nsw.show()
valid_flat_nsw.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+--------------------+--------------------+--------------------+---------------+------------------+--------+
|   Subs_Id|hour|         sector_name|  event_code_feature|           cell_name|          state|       utilisation|trace_id|
+----------+----+--------------------+--------------------+--------------------+---------------+------------------+--------+
|2100006261|  01|202583_RandwickCe...|     (30,[13],[1.0])|202583_RandwickCe...|New South Wales|10.772307692307692|       1|
|2100006261|  04|202583_RandwickCe...|      (30,[8],[1.0])|202583_RandwickCe...|New South Wales|12.544615384615383|       2|
|2100006261|  07|202583_RandwickCe...|     (30,[13],[1.0])|202583_RandwickCe...|New South Wales|24.883076923076924|       3|
|2100006261|  10|202583_RandwickCe...|      (30,[8],[1.0])|202583_RandwickCe...|New South Wales|25.113846153846154|       4|
|2100006261|  12|202583_RandwickCe...|     (30,[13],[1.0])|202583_RandwickCe...|New South Wales|43.815384615384616|       5|


In [175]:
valid_flat_nsw = valid_flat_nsw.join(
    valid_cei_df,
    on=['Subs_Id', 'hour'],
    how='inner'
)
valid_flat_nsw.persist()
valid_flat_nsw.count()
# flat_nsw.count()  # 54790880

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

54790880

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
# valid_flat_nsw = valid_flat_nsw.withColumn('trace_id', f.col('trace_id') - 1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [165]:
valid_flat_nsw.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+------------------------+------------------------------------+-----------------------------+---------------+------------------+--------+-------------------------+
|Subs_Id   |hour|sector_name             |event_code_feature                  |cell_name                    |state          |utilisation       |trace_id|customer_experience_index|
+----------+----+------------------------+------------------------------------+-----------------------------+---------------+------------------+--------+-------------------------+
|2100006525|02  |201803_Penhust594FstRd_2|(30,[13],[4.0])                     |201803_Penhust594FstRd_L08A_2|New South Wales|48.32             |634     |86.0                     |
|2100006597|16  |202818_ManlyCorso_3     |(30,[7],[2.0])                      |202818_ManlyCorso_L08A_3     |New South Wales|26.2              |1028    |81.0                     |
|2100006597|16  |241168_ManlyHospTemp_2  |(30,[7,13],[1.0,1.0])               |241168_ManlyHospTemp_

In [26]:
valid_flat_nsw.groupBy('trace_id').count().orderBy(f.col('count').desc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+
|trace_id|count|
+--------+-----+
|48387790|   89|
|38532508|   86|
|14440774|   86|
|25019365|   85|
|51320267|   79|
| 7338319|   78|
|50123402|   75|
|16095169|   74|
|45354005|   73|
|17988659|   71|
|26784577|   70|
|15278281|   70|
|11448279|   70|
|48898415|   69|
|12075277|   69|
|45693110|   69|
|22933836|   68|
|14246668|   68|
|51950403|   68|
|49387053|   68|
+--------+-----+
only showing top 20 rows

In [27]:
valid_flat_nsw.select('trace_id', 'customer_experience_index').dropDuplicates().orderBy('trace_id').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------------+
|trace_id|customer_experience_index|
+--------+-------------------------+
|       0|                     97.0|
|       1|                     26.0|
|       2|                     93.0|
|       3|                     23.0|
|       4|                     87.0|
|       5|                     16.0|
|       6|                     86.0|
|       7|                     13.0|
|       9|                     70.0|
|      10|                     27.0|
|      11|                     17.0|
|      12|                     90.0|
|      13|                     19.0|
|      14|                     27.0|
|      15|                     98.0|
|      16|                     27.0|
|      17|                     17.0|
|      18|                     87.0|
|      19|                     55.0|
|      21|                     24.0|
+--------+-------------------------+
only showing top 20 rows

In [167]:
valid_flat_nsw.select(f.max('trace_id')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|max(trace_id)|
+-------------+
|     54855701|
+-------------+

In [ ]:
# revise trace id

In [176]:
tmp_trace_id = valid_flat_nsw.select('Subs_Id', 'hour').dropDuplicates().withColumn(
    'trace_id', f.row_number().over(Window.orderBy('Subs_Id', 'hour'))
)
tmp_trace_id.persist()
tmp_trace_id.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+--------+
|   Subs_Id|hour|trace_id|
+----------+----+--------+
|2100006261|  01|       1|
|2100006261|  04|       2|
|2100006261|  07|       3|
|2100006261|  10|       4|
|2100006261|  12|       5|
|2100006261|  15|       6|
|2100006261|  17|       7|
|2100006261|  20|       8|
|2100006274|  00|       9|
|2100006274|  03|      10|
|2100006274|  08|      11|
|2100006274|  10|      12|
|2100006274|  13|      13|
|2100006285|  01|      14|
|2100006285|  02|      15|
|2100006285|  05|      16|
|2100006285|  11|      17|
|2100006285|  13|      18|
|2100006285|  16|      19|
|2100006285|  17|      20|
+----------+----+--------+
only showing top 20 rows

In [177]:
tmp_trace_id.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14965018

In [178]:
tmp_trace_id.select(f.max('trace_id')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|max(trace_id)|
+-------------+
|     14965018|
+-------------+

In [186]:
valid_flat_nsw = valid_flat_nsw.withColumn(
    'trace_id', f.dense_rank().over(Window.orderBy('Subs_Id', 'hour')) - 1
)
valid_flat_nsw.select(f.min('trace_id'), f.max('trace_id')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------------+
|min(trace_id)|max(trace_id)|
+-------------+-------------+
|            0|     14965017|
+-------------+-------------+

In [187]:
valid_flat_nsw.write.mode('overwrite').parquet(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/valid_flat_nsw/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [189]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Trace INFO

In [5]:
valid_flat_nsw = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/valid_flat_nsw/')
valid_flat_nsw.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

54790880

In [191]:
cei_threshold = 70

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [192]:
valid_flat_nsw.select(
    'trace_id', 'customer_experience_index'
).dropDuplicates().withColumn(
    'trace_bool', f.col('customer_experience_index') >= cei_threshold
).groupBy('trace_bool').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+
|trace_bool|   count|
+----------+--------+
|      true|12005748|
|     false| 2959270|
+----------+--------+

In [193]:
trace_info = valid_flat_nsw.select(
    'trace_id', 'customer_experience_index'
).dropDuplicates().orderBy('trace_id').withColumn(
    'trace_bool', (f.col('customer_experience_index') >= cei_threshold).cast('int')
).withColumn(
    'error_trace_type', f.col('trace_bool')
)
trace_info.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------------+----------+----------------+
|trace_id|customer_experience_index|trace_bool|error_trace_type|
+--------+-------------------------+----------+----------------+
|       0|                     97.0|         1|               1|
|       1|                     26.0|         0|               0|
|       2|                     93.0|         1|               1|
|       3|                     23.0|         0|               0|
|       4|                     87.0|         1|               1|
|       5|                     16.0|         0|               0|
|       6|                     86.0|         1|               1|
|       7|                     13.0|         0|               0|
|       8|                     70.0|         1|               1|
|       9|                     27.0|         0|               0|
|      10|                     17.0|         0|               0|
|      11|                     90.0|         1|               1|
|      12|               

In [194]:
trace_info.repartition(1).write.mode('overwrite').parquet(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/trace_info/')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Node ID and Type

In [6]:
@f.udf(returnType=ArrayType(ArrayType(StringType())))
def construct_nodes(edge_input):
    """
    node types:
    0 - user
    1 - cell
    2 - sector
    """
    node_list = []
    for a in edge_input:
        if [a[0], 0] not in node_list:
            node_list.append([a[0], 0])
        if [a[1], 1] not in node_list:
            node_list.append([a[1], 1])
        if [a[2], 2] not in node_list:
            node_list.append([a[2], 2])
    return node_list


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [223]:
valid_flat_nsw.groupBy('trace_id').agg(
    construct_nodes(
            f.collect_list(f.struct('Subs_Id', 'cell_name', 'sector_name'))
        ).alias('node_list')
).withColumn(
    'node_id_type', f.explode('node_list')
).drop('node_list').withColumn(
    'node_name', f.col('node_id_type')[0]
).withColumn(
    'node_type', f.col('node_id_type')[1]
).drop('node_id_type').withColumn(
    'node_id', f.row_number().over(Window.partitionBy('trace_id').orderBy('node_type')) - 1
).write.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_overview/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
node_overview = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_overview/')
node_overview.count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

110408369

In [8]:
node_overview.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------+-------+
|trace_id|           node_name|node_type|node_id|
+--------+--------------------+---------+-------+
|      65|          2100006360|        0|      0|
|      65|240028_Telarah_L1...|        1|      1|
|      65|    240028_Telarah_2|        2|      2|
|     133|          2100006452|        0|      0|
|     133|202647_Berrima_L0...|        1|      1|
|     133|    202647_Berrima_1|        2|      2|
|     148|          2100006452|        0|      0|
|     148|202647_Berrima_L1...|        1|      1|
|     148|202647_Berrima_L0...|        1|      2|
|     148|    202647_Berrima_1|        2|      3|
+--------+--------------------+---------+-------+
only showing top 10 rows

### Edge Index

In [9]:
valid_flat_nsw.filter(f.col('trace_id') == 54790878).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+-----------+------------------+---------+-----+-----------+--------+-------------------------+
|Subs_Id|hour|sector_name|event_code_feature|cell_name|state|utilisation|trace_id|customer_experience_index|
+-------+----+-----------+------------------+---------+-----+-----------+--------+-------------------------+
+-------+----+-----------+------------------+---------+-----+-----------+--------+-------------------------+

In [10]:
@f.udf(returnType=ArrayType(ArrayType(StringType())))
def construct_edge_index(edge_input):

    edge_index = []
    for a in edge_input:
        if [a[0], a[1]] not in edge_index:
            edge_index.append([a[0], a[1]])
        if [a[1], a[2]] not in edge_index:
            edge_index.append([a[1], a[2]])
    return edge_index

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
edge_index = valid_flat_nsw.groupBy('trace_id').agg(
        construct_edge_index(
            f.collect_list(f.struct('Subs_Id', 'cell_name', 'sector_name'))
        ).alias('edge_index_list')
).withColumn(
    'edge_pair', f.explode('edge_index_list')
).drop('edge_index_list').withColumn(
    'src_name', f.col('edge_pair')[0]
).withColumn(
    'dst_name', f.col('edge_pair')[1]
).drop('edge_pair')

edge_index.persist()
edge_index.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+
|trace_id|            src_name|            dst_name|
+--------+--------------------+--------------------+
|     148|          2100006452|202647_Berrima_L1...|
|     148|202647_Berrima_L1...|    202647_Berrima_1|
|     148|          2100006452|202647_Berrima_L0...|
|     148|202647_Berrima_L0...|    202647_Berrima_1|
|     463|          2100006643|228254_Mullumbimb...|
|     463|228254_Mullumbimb...|228254_Mullumbimb...|
|     471|          2100006643|228254_Mullumbimb...|
|     471|228254_Mullumbimb...|228254_Mullumbimb...|
|     471|          2100006643|202623_Mullumbimb...|
|     471|202623_Mullumbimb...|202623_Mullumbimby_2|
|     471|          2100006643|228254_Mullumbimb...|
|     471|228254_Mullumbimb...|228254_Mullumbimb...|
|     471|          2100006643|228254_Mullumbimb...|
|     471|228254_Mullumbimb...|228254_Mullumbimb...|
|     496|          2100006656|201885_TheEntranc...|
|     496|201885_TheEntranc...|201885_TheEntra

In [12]:
edge_index.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109581760

In [31]:
spark.conf.set('spark.default.parallelism', 1200)
spark.conf.set('spark.sql.shuffle.partitions', 1200)
spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '-1')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
edge_index.join(
    node_overview.select(
        'trace_id', 'node_name', 'node_id').withColumnRenamed(
        'node_name', 'src_name').withColumnRenamed('node_id', 'src_id'),
    on=['trace_id', 'src_name'],
    how='inner'
).join(node_overview.select(
        'trace_id', 'node_name', 'node_id').withColumnRenamed(
        'node_name', 'dst_name').withColumnRenamed('node_id', 'dst_id'),
    on=['trace_id', 'dst_name'],
    how='inner'
).drop('dst_name', 'src_name').write.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/edge_index/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Node Features

In [23]:
valid_flat_nsw.select(
    'trace_id', 'Subs_Id', 'cell_name', 'sector_name', 'event_code_feature', 'utilisation'
).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+-----------------------------+------------------------+---------------------+------------------+
|trace_id|Subs_Id   |cell_name                    |sector_name             |event_code_feature   |utilisation       |
+--------+----------+-----------------------------+------------------------+---------------------+------------------+
|0       |2100006261|202583_RandwickCemetry_L21H_1|202583_RandwickCemetry_1|(30,[13],[1.0])      |10.772307692307692|
|1       |2100006261|202583_RandwickCemetry_L21C_1|202583_RandwickCemetry_1|(30,[8],[1.0])       |12.544615384615383|
|2       |2100006261|202583_RandwickCemetry_L21C_1|202583_RandwickCemetry_1|(30,[13],[1.0])      |24.883076923076924|
|3       |2100006261|202583_RandwickCemetry_L21C_1|202583_RandwickCemetry_1|(30,[8],[1.0])       |25.113846153846154|
|4       |2100006261|202583_RandwickCemetry_L18A_1|202583_RandwickCemetry_1|(30,[13],[1.0])      |43.815384615384616|
|5       |2100006261|202583_RandwickCemetry_L21C_1|20258

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
subs_feature = valid_flat_nsw.select(
    'trace_id', 'Subs_Id', 'cell_name', 'sector_name', 'event_code_feature', 'utilisation'
).groupBy('trace_id', 'Subs_Id').agg(
    sparse_vector_sum(f.collect_list('event_code_feature')).alias('event_code_feature')
).withColumn(
    'feature', to_dense(f.col('event_code_feature'))
).drop('event_code_feature')
subs_feature.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14965018

In [37]:
subs_feature.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|trace_id|Subs_Id   |feature                                                                                                                                                       |
+--------+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|105     |2100006443|[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   |
|156     |2100006459|[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 37.0, 18.0, 0.0, 0.0, 1.0, 0.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]|
|188     |2100006496|[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0

In [34]:
cell_feature = valid_flat_nsw.select(
    'trace_id', 'Subs_Id', 'cell_name', 'sector_name', 'event_code_feature', 'utilisation'
).groupBy('trace_id', 'cell_name').agg(
    sparse_vector_sum(f.collect_list('event_code_feature')).alias('event_code_feature')
).withColumn(
    'feature', to_dense(f.col('event_code_feature'))
).drop('event_code_feature')
cell_feature.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

54790880

In [36]:
cell_feature.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|trace_id|cell_name                   |feature                                                                                                                                                    |
+--------+----------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|151     |228122_WarwickFarm_L18A_3   |[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]|
|276     |227535_Jannali_L18B_3       |[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]|
|289     |227535_Jan

In [42]:
@f.udf(returnType=ArrayType(FloatType()))
def overwrite_utilisation(v, util):
    v[-1] = util
    return v

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
sector_feature = valid_flat_nsw.select(
    'trace_id', 'Subs_Id', 'cell_name', 'sector_name', 'event_code_feature', 'utilisation'
).groupBy('trace_id', 'sector_name').agg(
    sparse_vector_sum(f.collect_list('event_code_feature')).alias('event_code_feature'),
    f.max('utilisation').alias('utilisation')
).withColumn(
    'feature', to_dense(f.col('event_code_feature'))
).drop('event_code_feature').withColumn(
    'feature', overwrite_utilisation(f.col('feature'), f.col('utilisation'))
).drop('utilisation')
sector_feature.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

40652471

In [46]:
sector_feature.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|trace_id|sector_name             |feature                                                                                                                                                          |
+--------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|132     |202647_Berrima_1        |[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.59]     |
|221     |201656_CabbageTrlnePrk_1|[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 59.73]    |
|258     |

In [51]:
node_features = subs_feature.withColumnRenamed(
    'Subs_Id', 'node_name'
).union(
    cell_feature.withColumnRenamed('cell_name', 'node_name')
).union(
    sector_feature.withColumnRenamed('sector_name', 'node_name')
).join(
    node_overview.select('trace_id', 'node_name', 'node_id'),
    on=['trace_id', 'node_name'],
    how='inner'
)

for i in range(31):
    node_features = node_features.withColumn(f'f_{i}', f.col('feature')[i])

node_features = node_features.drop('feature')

node_features.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+
|trace_id|           node_name|node_id|f_0|f_1|f_2|f_3|f_4|f_5|f_6|f_7|f_8|f_9|f_10|f_11|f_12|f_13|f_14|f_15|f_16|f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|     f_30|
+--------+--------------------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+
|     414|201775_EastHills_...|      1|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|      0.0|
|     471|228254_Mullumbimb...|      7|0.0|0.0|0.0|0.0|0.0|0.0|0.0|1.0|5.0|0.0| 0.0| 3.0| 4.0|21.0| 0.0| 0.0| 2.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|    61.32|
|     608|  202930_Roseberry_3|      8|0.0|0.0|0.0

In [ ]:
node_features.drop('node_name').write.parquet(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_features/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
node_features = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_features/')
node_features.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+
|trace_id|node_id|f_0|f_1|f_2|f_3|f_4|f_5|f_6|f_7|f_8|f_9|f_10|f_11|f_12|f_13|f_14|f_15|f_16|f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|    f_30|
+--------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+
|     361|     29|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0| 1.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|44.27077|
|     418|      4|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0| 0.0| 0.0| 0.0|10.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|41.51077|
|     431|      2|0.0|0.0|0.0|0.0|0.0|0.0|0.0|7.0|0.0|0.0| 0.0| 0.0| 0.0| 5.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|  

In [7]:
node_features.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

110408369

In [6]:
node_overview = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_overview/')
node_overview.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

110408369

In [8]:
node_overview.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------------------------+---------+-------+
|trace_id|node_name                   |node_type|node_id|
+--------+----------------------------+---------+-------+
|75      |2100006363                  |0        |0      |
|75      |201645_PaddingtonNth_L21H_3 |1        |1      |
|75      |201645_PaddingtonNth_3      |2        |2      |
|195     |2100006504                  |0        |0      |
|195     |222122_LaPerouse_L18B_1     |1        |1      |
|195     |228289_AirportTunlWst_L21C_1|1        |2      |
|195     |227563_SaltPanCreek_L08A_1  |1        |3      |
|195     |227544_RevesbyHeights_L08A_3|1        |4      |
|195     |227476_MaroubraBeach_L18B_1 |1        |5      |
|195     |202125_Padstow_L08A_3       |1        |6      |
+--------+----------------------------+---------+-------+
only showing top 10 rows

### Node Type

In [12]:
node_overview.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trace_id: integer (nullable = true)
 |-- node_name: string (nullable = true)
 |-- node_type: string (nullable = true)
 |-- node_id: integer (nullable = true)

In [13]:
@f.udf(returnType=ArrayType(ArrayType(IntegerType())))
def construct_node_types(node_list):
    node_types = [[] for i in range(3)]
    for ntype, nid in node_list:
        node_types[int(ntype)].append(nid)
    return node_types
        

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
node_types = node_overview.groupBy('trace_id').agg(
    construct_node_types(
        f.collect_list(f.struct('node_type', 'node_id'))
    ).alias('node_types')
).orderBy('trace_id')
node_types.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14965018

In [18]:
node_types.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+
|trace_id|          node_types|
+--------+--------------------+
|       0|     [[0], [1], [2]]|
|       1|     [[0], [1], [2]]|
|       2|     [[0], [1], [2]]|
|       3|     [[0], [1], [2]]|
|       4|     [[0], [1], [2]]|
|       5|     [[0], [1], [2]]|
|       6|     [[0], [1], [2]]|
|       7|     [[0], [1], [2]]|
|       8|     [[0], [1], [2]]|
|       9|     [[0], [1], [2]]|
|      10|     [[0], [1], [2]]|
|      11|     [[0], [1], [2]]|
|      12|     [[0], [1], [2]]|
|      13|     [[0], [1], [2]]|
|      14|     [[0], [1], [2]]|
|      15|     [[0], [1], [2]]|
|      16|     [[0], [1], [2]]|
|      17|     [[0], [1], [2]]|
|      18|[[0], [1, 2], [3,...|
|      19|     [[0], [1], [2]]|
+--------+--------------------+
only showing top 20 rows

In [19]:
node_types.write.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_types/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## All Data

In [6]:
edge_index = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/edge_index/')
# node_features = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_features/')
node_features = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_features_imputed/')
node_types = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_types/')
trace_info = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/trace_info/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
print(edge_index.count())
print(node_features.count())
print(node_types.count())
print(trace_info.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109581760
110408369
14965018
14965018

In [30]:
edge_index.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------+
|trace_id|src_id|dst_id|
+--------+------+------+
|     103|     0|     2|
|     197|     0|     2|
|     197|     0|    21|
|     266|     1|     5|
|     275|     0|     1|
|     290|     0|     2|
|     301|     1|     3|
|     368|     2|     5|
|     421|     2|     6|
|     424|     6|    13|
+--------+------+------+
only showing top 10 rows

In [29]:
trace_info.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------------+----------+----------------+
|trace_id|customer_experience_index|trace_bool|error_trace_type|
+--------+-------------------------+----------+----------------+
|  584395|                     90.0|         1|               1|
|  584396|                     71.0|         1|               1|
|  584397|                     76.0|         1|               1|
|  584398|                     83.0|         1|               1|
|  584399|                     86.0|         1|               1|
|  584400|                     83.0|         1|               1|
|  584401|                     66.0|         0|               0|
|  584402|                     74.0|         1|               1|
|  584403|                     69.0|         0|               0|
|  584404|                     87.0|         1|               1|
+--------+-------------------------+----------+----------------+
only showing top 10 rows

### Edge Ratio

In [38]:
_edge_ratio = edge_index.join(
    trace_info.select('trace_id', 'trace_bool'),
    on='trace_id',
    how='inner'
).join(
    node_overview.select(
        'trace_id', f.col('node_id').alias('src_id'), f.col('node_type').alias('src_type')
    ),
    on=['trace_id', 'src_id'],
    how='inner'
).join(
    node_overview.select(
        'trace_id', f.col('node_id').alias('dst_id'), f.col('node_type').alias('dst_type')
    ),
    on=['trace_id', 'dst_id'],
    how='inner'
)

_edge_ratio.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------+----------+--------+--------+
|trace_id|dst_id|src_id|trace_bool|src_type|dst_type|
+--------+------+------+----------+--------+--------+
|   83102|    52|    23|         1|       1|       2|
|   83122|     9|     6|         1|       1|       2|
|  115121|     1|     0|         1|       0|       1|
|  199212|     9|     0|         0|       0|       1|
|  260194|     4|     1|         1|       1|       2|
|  266436|     5|     2|         1|       1|       2|
|  266818|     2|     0|         1|       0|       1|
|  300803|     6|     1|         1|       1|       2|
|  301302|     2|     1|         1|       1|       2|
|  339114|     2|     1|         0|       1|       2|
|  381257|    16|     0|         1|       0|       1|
|  404748|    36|    13|         1|       1|       2|
|  553083|     2|     1|         1|       1|       2|
|  573787|     2|     0|         1|       0|       1|
|  685247|     6|     2|         1|       1|       2|
|  696043|     7|     4|    

In [41]:
_edge_ratio_all = _edge_ratio.groupBy('trace_bool', 'trace_id').count()
_edge_ratio_all.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+-----+
|trace_bool|trace_id|count|
+----------+--------+-----+
|         1| 6150958|   20|
|         1| 6102269|   28|
|         1|14282488|    2|
|         1|10300662|   30|
|         1|12931294|    6|
+----------+--------+-----+
only showing top 5 rows

In [42]:
_edge_ratio_type = _edge_ratio.withColumn(
    'src_dst_type', f.concat('src_type', f.lit('_'), 'dst_type')
).groupBy('trace_bool', 'trace_id', 'src_dst_type').count()

_edge_ratio_type.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+------------+-----+
|trace_bool|trace_id|src_dst_type|count|
+----------+--------+------------+-----+
|         1|11314085|         1_2|   39|
|         0|11073042|         1_2|    4|
|         1|  873426|         1_2|    1|
|         1| 2280413|         1_2|   10|
|         0|14332479|         1_2|    3|
+----------+--------+------------+-----+
only showing top 5 rows

In [49]:
_edge_ratio_type.join(
    _edge_ratio_all.withColumnRenamed('count', 'total'),
    on=['trace_bool', 'trace_id'],
    how='inner'
).withColumn(
    'percent_in_all', 1.0 * f.col('count') / f.col('total')
).groupBy(
    'trace_bool', 'src_dst_type'
).agg(
    f.percentile_approx('percent_in_all', 0.95, f.lit(1000000)).alias('percent_in_all')
).withColumn(
    'edge_type', f.lit(0)
).withColumn(
    'level_3', f.lit(0.95)
).write.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/edge_ratio/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
edge_ratio = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/edge_ratio/')
edge_ratio.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------------+---------+-------+
|trace_bool|src_dst_type|percent_in_all|edge_type|level_3|
+----------+------------+--------------+---------+-------+
|         0|         0_1|           0.5|        0|   0.95|
|         0|         1_2|           0.5|        0|   0.95|
|         1|         1_2|           0.5|        0|   0.95|
|         1|         0_1|           0.5|        0|   0.95|
+----------+------------+--------------+---------+-------+

In [70]:
edge_ratio = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/edge_ratio/')
edge_ratio.repartition(1).write.csv('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/csv/edge_ratio/', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Edge Index

In [8]:
edge_index.repartition(1).write.mode('overwrite').csv(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/csv/edge_index/', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### node features

In [24]:
node_features.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----------------+
|trace_id|node_id|f_0|f_1|f_2|f_3|f_4|f_5|f_6|f_7|f_8|f_9|f_10|f_11|f_12|f_13|f_14|f_15|f_16|f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|             f_30|
+--------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----------------+
|   87616|      2|0.0|0.0|0.0|0.0|0.0|1.0|0.0|0.0|0.0|0.0| 0.0| 1.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|27.94769287109375|
|  610777|     14|0.0|0.0|0.0|0.0|0.0|0.0|0.0|1.0|1.0|0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|              0.0|
|  610777|     20|0.0|0.0|0.0|0.0|0.0|0.0|0.0|3.0|0.0|0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0

In [10]:
node_features.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trace_id: integer (nullable = true)
 |-- node_id: integer (nullable = true)
 |-- f_0: float (nullable = true)
 |-- f_1: float (nullable = true)
 |-- f_2: float (nullable = true)
 |-- f_3: float (nullable = true)
 |-- f_4: float (nullable = true)
 |-- f_5: float (nullable = true)
 |-- f_6: float (nullable = true)
 |-- f_7: float (nullable = true)
 |-- f_8: float (nullable = true)
 |-- f_9: float (nullable = true)
 |-- f_10: float (nullable = true)
 |-- f_11: float (nullable = true)
 |-- f_12: float (nullable = true)
 |-- f_13: float (nullable = true)
 |-- f_14: float (nullable = true)
 |-- f_15: float (nullable = true)
 |-- f_16: float (nullable = true)
 |-- f_17: float (nullable = true)
 |-- f_18: float (nullable = true)
 |-- f_19: float (nullable = true)
 |-- f_20: float (nullable = true)
 |-- f_21: float (nullable = true)
 |-- f_22: float (nullable = true)
 |-- f_23: float (nullable = true)
 |-- f_24: float (nullable = true)
 |-- f_25: float (nullable = true)
 |-- f_26: flo

In [11]:
# node_features.describe().repartition(1).write.mode('overwrite').csv('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/csv/summary/')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinMaxScaler, VectorAssembler

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
assembler = VectorAssembler(inputCols=[f'f_{i}' for i in range(31)], outputCol='feature_vector')
scaler = MinMaxScaler(inputCol='feature_vector', outputCol='feature_scaled', )

pipeline = Pipeline(
    stages=[assembler, scaler]
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
scaled_features = pipeline.fit(node_features).transform(node_features).select('trace_id', 'node_id', 'feature_scaled')
# scaled_features.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
node_feature_norm = scaled_features.withColumn(
    'feature_list', to_dense_flat('feature_scaled')
)
# node_feature_norm.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
# for col in node_features.columns:
#     print(col)
#     node_features.filter(f.col(col).isNull()).show()
#     print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
for i in range(31):
    node_feature_norm = node_feature_norm.withColumn(f'f_{i}', f.col('feature_list')[i])
node_feature_norm = node_feature_norm.drop('feature_scaled', 'feature_list')
node_feature_norm.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---+---+---+---+---+-----------+---+------------+------------+---+----+------------+----+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----------+
|trace_id|node_id|f_0|f_1|f_2|f_3|f_4|f_5        |f_6|f_7         |f_8         |f_9|f_10|f_11        |f_12|f_13        |f_14|f_15|f_16|f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|f_30      |
+--------+-------+---+---+---+---+---+-----------+---+------------+------------+---+----+------------+----+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----------+
|87616   |2      |0.0|0.0|0.0|0.0|0.0|0.004672897|0.0|0.0         |0.0         |0.0|0.0 |0.0070422534|0.0 |6.2189053E-4|0.0 |0.0 |0.0 |0.5 |0.5 |0.5 |0.5 |0.5 |0.0 |0.0 |0.0 |0.0 |0.0 |0.5 |0.5 |0.5 |0.28071207|
|610777  |14     |0.0|0.0|0.0|0.0|0.0|0.0        |0.0|1.9278967E-4|3.08642E-4  |0.0|0.0 |0.0         |0.0 |0.0         |0.0 |0.0 |0.0 |0.5 |0.5 |0.5 |0.

In [180]:
node_features.filter('trace_id = 87616').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----------------+
|trace_id|node_id|f_0|f_1|f_2|f_3|f_4|f_5|f_6|f_7|f_8|f_9|f_10|f_11|f_12|f_13|f_14|f_15|f_16|f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|             f_30|
+--------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----------------+
|   87616|      2|0.0|0.0|0.0|0.0|0.0|1.0|0.0|0.0|0.0|0.0| 0.0| 1.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|27.94769287109375|
|   87616|      0|0.0|0.0|0.0|0.0|0.0|1.0|0.0|0.0|0.0|0.0| 0.0| 1.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|              0.0|
|   87616|      1|0.0|0.0|0.0|0.0|0.0|1.0|0.0|0.0|0.0|0.0| 0.0| 1.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0

In [181]:
node_feature_norm.filter('trace_id = 87616').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---+---+---+---+---+-----------+---+---+---+---+----+------------+----+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----------+
|trace_id|node_id|f_0|f_1|f_2|f_3|f_4|        f_5|f_6|f_7|f_8|f_9|f_10|        f_11|f_12|        f_13|f_14|f_15|f_16|f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|      f_30|
+--------+-------+---+---+---+---+---+-----------+---+---+---+---+----+------------+----+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----------+
|   87616|      2|0.0|0.0|0.0|0.0|0.0|0.004672897|0.0|0.0|0.0|0.0| 0.0|0.0070422534| 0.0|6.2189053E-4| 0.0| 0.0| 0.0| 0.5| 0.5| 0.5| 0.5| 0.5| 0.0| 0.0| 0.0| 0.0| 0.0| 0.5| 0.5| 0.5|0.28071207|
|   87616|      0|0.0|0.0|0.0|0.0|0.0|0.004672897|0.0|0.0|0.0|0.0| 0.0|0.0070422534| 0.0|6.2189053E-4| 0.0| 0.0| 0.0| 0.5| 0.5| 0.5| 0.5| 0.5| 0.0| 0.0| 0.0| 0.0| 0.0| 0.5| 0.5| 0.5|       0.0|
|   87616|      1|0.0|0.0|0.0|

In [21]:
node_feature_norm.write.mode('overwrite').parquet(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_feature_norm/')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
node_feature_norm = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_feature_norm/')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
node_feature_norm.show()
node_feature_norm.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---+---+---+---+---+---+---+------------+----------+------------+------------+----+----+------------+----+----+-----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----------+
|trace_id|node_id|f_0|f_1|f_2|f_3|f_4|f_5|f_6|         f_7|       f_8|         f_9|        f_10|f_11|f_12|        f_13|f_14|f_15|       f_16|f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|      f_30|
+--------+-------+---+---+---+---+---+---+---+------------+----------+------------+------------+----+----+------------+----+----+-----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----------+
|   28682|      1|0.0|0.0|0.0|0.0|0.0|0.0|0.0|         0.0|       0.0|         0.0|         0.0| 0.0| 0.0|0.0074626864| 0.0| 0.0|        0.0| 0.5| 0.5| 0.5| 0.5| 0.5| 0.0| 0.0| 0.0| 0.0| 0.0| 0.5| 0.5| 0.5|       0.0|
|  146451|      0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0042413725|       0.0|         0.0|         0.0| 0.0| 0.0|0.0031094528| 0.0| 0.0

In [59]:
_cols = node_feature_norm.columns


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
len(_cols)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

33

In [61]:
node_feature_norm.repartition(1).write.mode('overwrite').csv('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/csv/node_feature_norm/', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Node Type

In [69]:
node_types.orderBy('trace_id').select(f.col('node_types').cast('string')).repartition(1).write.mode('overwrite').csv(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/csv/node_types/', header=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
spark.read.csv('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/csv/node_types/').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|                 _c0|
+--------------------+
|          node_types|
|     [[0], [1], [2]]|
|     [[0], [1], [2]]|
|     [[0], [1], [2]]|
|     [[0], [1], [2]]|
|     [[0], [1], [2]]|
|     [[0], [1], [2]]|
|[[0], [1, 2, 3, 4...|
|  [[0], [1, 2], [3]]|
|  [[0], [1, 2], [3]]|
|[[0], [1, 2, 3, 4...|
|[[0], [1, 2, 3, 4...|
|[[0], [1, 2, 3, 4...|
|[[0], [1, 2, 3], ...|
|[[0], [1, 2, 3, 4...|
|     [[0], [1], [2]]|
|[[0], [1, 2, 3, 4...|
|[[0], [1, 2, 3, 4...|
|[[0], [1, 2, 3], ...|
|[[0], [1, 2, 3], ...|
+--------------------+
only showing top 20 rows

### Trace Info

In [72]:
trace_info.repartition(1).write.csv(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/csv/trace_info/', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Impute missing utilisation

In [145]:
valid_traces = node_features.filter(f.col('f_30').isNotNull()).select('trace_id').dropDuplicates()
valid_traces = valid_traces.withColumn('index', f.row_number().over(Window.orderBy('trace_id')) - 1)
valid_traces.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14965018

In [119]:
valid_traces.select(f.max('trace_id'), f.max('index')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+
|max(trace_id)|max(index)|
+-------------+----------+
|     14965017|  14965017|
+-------------+----------+

In [115]:
# trace_info
trace_info.join(
    valid_traces,
    on='trace_id'
).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14965018

In [120]:
node_features.select(f.max('trace_id')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|max(trace_id)|
+-------------+
|     14965017|
+-------------+

In [146]:
all_null_trace_id = node_features.filter(f.col('f_30').isNull()).select('trace_id')
all_null_trace_id.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|trace_id|
+--------+
|  791653|
| 1422105|
| 2022641|
| 3368644|
| 3379162|
| 4264541|
| 4668436|
| 4847498|
| 5145178|
| 5279908|
| 5514397|
| 6797104|
| 7258190|
| 7459787|
|10363567|
|10377699|
|10471067|
|11310819|
|11543406|
|12386273|
+--------+
only showing top 20 rows

In [138]:
# null_trace_id_mean = None
# for t in all_null_trace_id:
#     if null_trace_id_mean is None:
#         null_trace_id_mean = node_features.filter(f.col('trace_id') == t).groupBy('trace_id').agg(f.mean('f_30').alias('mean_util'))
#     else:
#         null_trace_id_mean.union(node_features.filter(f.col('trace_id') == t).groupBy('trace_id').agg(f.mean('f_30').alias('mean_util')))

# null_trace_id_mean.show()

An error was encountered:
Session 0 did not reach idle status in time. Current status is busy.


In [150]:
null_trace_id_mean = node_features.join(
    all_null_trace_id,
    on='trace_id'
).groupBy('trace_id').agg(f.mean('f_30').alias('mean_util'))

null_trace_id_mean.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------+
|trace_id|         mean_util|
+--------+------------------+
| 2173297|15.716219820295061|
|13312270|17.215518246526305|
| 1428873|23.797460465204146|
| 9474347| 23.06367235183716|
| 4594806|10.566285542079381|
+--------+------------------+
only showing top 5 rows

In [156]:
node_features.join(
    null_trace_id_mean,
    on='trace_id',
    how='left'
).withColumn('f_30', f.coalesce('f_30', 'mean_util')).drop('mean_util').write.mode('overwrite').parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_features_imputed/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
node_features.filter(f.col('trace_id') == 791653).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+
|trace_id|node_id|f_0|f_1|f_2|f_3|f_4|f_5|f_6|f_7|f_8|f_9|f_10|f_11|f_12|f_13|f_14|f_15|f_16|f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|    f_30|
+--------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+
|  791653|      3|0.0|0.0|1.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|    null|
|  791653|      4|0.0|0.0|0.0|0.0|0.0|0.0|1.0|0.0|1.0|0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|50.01091|
|  791653|      2|0.0|0.0|0.0|0.0|0.0|0.0|1.0|0.0|1.0|0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|  

In [159]:
node_features_imputed.filter(f.col('trace_id') == 791653).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------------------+
|trace_id|node_id|f_0|f_1|f_2|f_3|f_4|f_5|f_6|f_7|f_8|f_9|f_10|f_11|f_12|f_13|f_14|f_15|f_16|f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|              f_30|
+--------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------------------+
|  791653|      3|0.0|0.0|1.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|12.502727508544922|
|  791653|      4|0.0|0.0|0.0|0.0|0.0|0.0|1.0|0.0|1.0|0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 50.01091003417969|
|  791653|      2|0.0|0.0|0.0|0.0|0.0|0.0|1.0|0.0|1.0|0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0

In [157]:
node_features_imputed = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_features_imputed/')
node_features_imputed.count(), node_features.count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(110408369, 110408369)